# Therapist and Child Detection and Tracking

# Import necessary libraries

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub


In [2]:
# Load the pre-trained SSD MobileNet model from TensorFlow Hub
model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

In [3]:
# Define a simple category index for person detection
category_index = {1: 'person'}


In [4]:
def detect_objects(frame):
    # Convert frame to tensor
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]
    
    # Run detection
    detections = model(input_tensor)
    
    return detections

def draw_boxes(frame, detections):
    for i in range(len(detections['detection_boxes'][0])):
        if detections['detection_scores'][0][i].numpy() > 0.5:
            box = detections['detection_boxes'][0][i].numpy()
            (startY, startX, endY, endX) = box
            startX = int(startX * frame.shape[1])
            startY = int(startY * frame.shape[0])
            endX = int(endX * frame.shape[1])
            endY = int(endY * frame.shape[0])

            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            label = category_index[1] + ": " + str(round(detections['detection_scores'][0][i].numpy(), 2))
            cv2.putText(frame, label, (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame

def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        detections = detect_objects(frame)
        frame = draw_boxes(frame, detections)

        out.write(frame)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [5]:
# Process the downloaded video
input_video_path = 'test_video.mp4'  
output_video_path = 'output_video.avi'  
process_video(input_video_path, output_video_path)

In [6]:
def detect_objects(frame):
    # Convert frame to tensor
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]
    
    # Run detection
    detections = model(input_tensor)
    
    return detections